### Unpack ETL files

In [ ]:
import os
from pathlib import Path

from openai.types.beta.threads import image_file
from pandas.core.common import random_state

project_root = Path().resolve()
etl_dir = project_root / "data" / "ETL8G"
unpack_script = project_root / "data" / "unpack_etlcdb" / "unpack_etlcdb" / "unpack.py"

files = os.listdir(etl_dir)

for file in files:
    if file != "ETL8INFO":
        input_file = etl_dir / file
        cmd = f'python {unpack_script} {input_file}'
        print("Running:", cmd)
        # os.system(cmd)



### Saving labels to list

In [ ]:
import pandas as pd

path_to_labels = Path().resolve()/ "data" / "ETL8G"/ "ETL8G_01_unpack"/"meta.csv"

labels_df = pd.read_csv(path_to_labels)

labels = labels_df["char"]
labels = labels.tolist()


In [ ]:
all_labels = labels.copy()
labels_copy = all_labels.copy()

for i in range(31):
    labels_copy_2 = labels_copy.copy()
    all_labels.extend(labels_copy_2)

all_labels.extend(labels_copy[:956])

In [ ]:
# go through each folder
path_label_list = []
etl_dir = Path().resolve() / "data" / "ETL8G"
for folder in os.listdir(etl_dir):
    if "unpack" in folder:
        folder_path = os.path.join(etl_dir, folder)

        # go through each png in folder
        for fname in os.listdir(folder_path):
            if fname.endswith(".png"):
                fpath = os.path.join(folder_path, fname)
                # take filename without extension
                idx = int(os.path.splitext(fname)[0])
                # compute label
                label = idx % 956
                path_label_list.append((fpath, label))

print(path_label_list[956])


In [ ]:
print(len(path_label_list))

In [ ]:
print(labels[1])
print(path_label_list[1912])

### Dataset class

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import torch
import torchvision.transforms as T

class ImageDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path, label = self.data[idx]
        img = Image.open(path).convert("L")
        if self.transform:
            img = self.transform(img)
        else:
            img = T.ToTensor()(img)
        return img, torch.tensor(label, dtype=torch.long)



In [ ]:
transform = T.Compose([
    T.Resize((64, 64)),
    T.ToTensor(),
    T.Normalize(mean=[0.5], std=[0.5]),
])

a = ImageDataset(path_label_list, transform)
img, lab = a[0]
# print(img.size)
print(lab)
print(type(img), img.shape, lab)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

img, label = a[5012]      # get transformed tensor (after __getitem__)
plt.imshow(img.squeeze(), cmap='gray')
plt.title(f"Label: {label}")
plt.axis('off')
plt.show()


### Splitting the data into train, val, test sets

In [ ]:
from sklearn.model_selection import train_test_split

train_data, temp_data = train_test_split(path_label_list, test_size=0.3, random_state=42, stratify=[d[1] for d in path_label_list]) # used stratify to preserve the same class proportions

val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=[d[1] for d in temp_data])


In [ ]:
train_dataset = ImageDataset(train_data, transform)
val_dataset   = ImageDataset(val_data, transform)
test_dataset  = ImageDataset(test_data, transform)


### Data Loader

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=32, num_workers=0)
test_loader  = DataLoader(test_dataset, batch_size=32, num_workers=0)


TEST

In [ ]:
imgs, labels = next(iter(train_loader))
print(imgs.shape)    # [32, 1, 64, 64]
print(labels.shape)  # [32]


### Model training

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

    def forward(self, x):
        return self.block(x)

In [ ]:
class CNN_Improved(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # 64x64 -> 32x32
        self.block1 = ConvBlock(1, 32)
        # 32x32 -> 16x16
        self.block2 = ConvBlock(32, 64)
        # 16x16 -> 8x8
        self.block3 = ConvBlock(64, 128)

        self.gap  = nn.AdaptiveAvgPool2d(1)
        self.drop = nn.Dropout(0.3)
        self.fc   = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.gap(x)
        x = torch.flatten(x, 1)
        x = self.drop(x)
        x = self.fc(x)
        return x

In [ ]:
import torch
import torch.nn as nn

@torch.no_grad()
def eval_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        batch_size = labels.size(0)
        total_loss += loss.item() * batch_size

        preds = outputs.argmax(1)
        total_correct += (preds == labels).sum().item()
        total_samples += batch_size

    avg_loss = total_loss / total_samples
    acc = total_correct / total_samples
    return avg_loss, acc


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

num_classes = 956
model = CNN_Improved(num_classes).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 50

train_losses = []
val_losses = []
val_accuracies = []

best_acc = 0.0
best_path = "best_model3.pt"
checkpoint_path = "cnn_kanji_checkpoint3.pt"



for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    total_train_samples = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        batch_size = labels.size(0)
        running_loss += loss.item() * batch_size
        total_train_samples += batch_size

    avg_train_loss = running_loss / total_train_samples


    val_loss, val_acc = eval_model(model, val_loader, criterion, device)

    train_losses.append(avg_train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"train_loss={avg_train_loss:.4f} | "
        f"val_loss={val_loss:.4f} | "
        f"val_acc={val_acc*100:.2f}%"
    )

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), best_path)
        print(f"✅ New best model saved (acc={best_acc:.4f})")

    checkpoint = {
        "epoch": epoch + 1,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "train_losses": train_losses,
        "val_losses": val_losses,
        "val_accuracies": val_accuracies,
        "best_acc": best_acc,
        # przydatne hiperparametry:
        "num_classes": num_classes,
        "learning_rate": 2e-3,
        "label_smoothing": 0.1,
        "model_name": "CNN_Improved_v1",
    }
    torch.save(checkpoint, checkpoint_path)

print("✅ Training complete!")
print(f"Best val_acc = {best_acc*100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(train_losses, label='Train Loss', marker='o')
plt.plot(val_losses, label='Validation Loss', marker='o')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# load weights
model = CNN_Improved(956)
model.load_state_dict(torch.load("best_model3.pt", map_location="cpu"))
model.eval()

In [ ]:
tfm = T.Compose([
    T.Resize((64,64)),
    T.ToTensor(),
    T.Normalize([0.5],[0.5])
])


In [ ]:
correct, total = 0, 0
test_loss = 0.0
loss_fn = torch.nn.CrossEntropyLoss()

with torch.no_grad():
    for x, y in test_loader:              # <- your existing DataLoader
        logits = model(x)
        test_loss += loss_fn(logits, y).item() * x.size(0)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += x.size(0)

print(f"Test acc: {100*correct/total:.2f}% | Test loss: {test_loss/total:.4f}")

In [ ]:
with torch.no_grad():
    logits = model(x)
    pred = logits.argmax(1).item()
    probs = F.softmax(logits, dim=1)
    confidence = probs[0, pred].item()

print(f"Predicted class index: {pred}  |  confidence: {confidence:.2f}")


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img, cmap="gray")
plt.title(f"Predicted class: {pred} (conf: {confidence:.2f})")
plt.axis("off")
plt.show()
